## Import essential packages

In [ ]:
#----- These are for the simulation 

from __future__ import division
import hoomd
import hoomd.md
import numpy
from matplotlib import pyplot

#----- These are for the orientation analysis

import gsd
import gsd.hoomd

# Simulation

In [ ]:
#-----Some important variables

box = 12.5
replicates = 10
p_max = 0.5
t_max = 1.0

In [ ]:
#-------Define the unit cell

sim = hoomd.context.initialize("");
uc = hoomd.lattice.unitcell(N = 1,
                            a1 = [box, 0, 0],
                            a2 = [0, box, 0],
                            a3 = [0, 0, box],
                            dimensions = 3,
                            position = [[0,0,0]],
                            type_name = ['M'],
                            mass = [1.0],
                            moment_inertia = [[0,
                                               1/12*1.0*8**2,
                                               1/12*1.0*8**2]],
                            orientation = [[1, 0, 0, 0]]);

#-----From the lattice we obtain a snapshot and initialize.

snapshot = uc.get_snapshot()
snapshot.replicate(replicates,replicates,replicates)
system = hoomd.init.read_snapshot(snapshot)

#-----Add the particle type for the constituent particles.

system.particles.types.add('A');

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.cell()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)


#-----Select an standard integrator

hoomd.md.integrate.mode_standard(dt=0.005);

#------Define group

mesogens = hoomd.group.rigid_center();

#----Integrate using NPT

pressure = hoomd.variant.linear_interp(points = [(0,1e-6), (1e4, p_max)])
temperature = hoomd.variant.linear_interp(points = [(0,0.5), (1e4, t_max)])

npt = hoomd.md.integrate.npt(group = mesogens, kT = temperature, tau = 0.5, tauP = 1.0, P = pressure);
npt.randomize_velocities(seed=42)


#-----Write output


gsd_file = "global_trayectory.gsd"

gsd = hoomd.dump.gsd(gsd_file,
               period=1e3,
               group=hoomd.group.all(),
               overwrite=True);  


log_file = "global_trayectory.log"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume',
                                      'npt_thermostat_energy'],
                         period=1e3,
                         overwrite=True);

## Run the simulation

In [ ]:
with sim:
    hoomd.run(6e4)
    npt.set_params(kT = 1.2)
    hoomd.run(6e4)
    npt.set_params(kT = 1.4)
    hoomd.run(6e4)
    npt.set_params(kT = 1.6)
    hoomd.run(6e4)
    npt.set_params(kT = 1.8)
    hoomd.run(6e4)
    npt.set_params(kT = 2.0)
    hoomd.run(6e4)

## System initial density

In [ ]:
particles = snapshot.particles.N

lx = snapshot.box.Lx
ly = snapshot.box.Ly
lz = snapshot.box.Lz
volume = lx * ly *lz

rho = particles / volume

print("Density = ", rho)

In [ ]:
system.get_metadata()